In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import shutil
import os
from tqdm.notebook import tqdm

# Define variables

In [ ]:
WEIGHTS = '/kaggle/input/vinbigdata-final-trainedmodels/yolov5x6.pt'
TEST_DIR = '/kaggle/input/vinbigdata-chest-xray-resized-png-1024x1024/vinbigdata/test'
TEST_METADATA_DIR = '/kaggle/input/vinbigdata-chest-xray-resized-png-1024x1024/vinbigdata/test'

In [ ]:
test_meta = pd.read_csv(TEST_METADATA_DIR)
test_meta.head()

In [ ]:
shutil.copytree('/kaggle/input/yolov5/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

# Inference

In [ ]:
!python detect.py --weights $WEIGHTS --img 1024 --conf 0.0001 --iou 0.5 --source $TEST_DIR --augment --save-txt --save-conf --exist-ok

# Format predictions

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float)
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
submission_list = []

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    width = test_meta.loc[test_meta['image_id'] == image_id, 'width'].values[0] 
    height = test_meta.loc[test_meta['image_id'] == image_id, 'height'].values[0] 
    
    f = open(file_path, 'r')    
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(height, width, data[:, 2:]), decimals=5)), axis =1).reshape(-1), decimals=5).astype(str))
    
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        
    final_string = ' '.join(bboxes)
    submission_list.append([image_id, final_string])

# Create submission file

In [ ]:
df = pd.DataFrame(submission_list, columns = ['image_id','PredictionString'])
df.to_csv("submission.csv",index=False)
df.head()